In [ ]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

from core.audio_model import AudioModel

In [ ]:
model_path = "../_models/speech_denoiser_model.pth"
model: AudioModel = torch.load(model_path, map_location='cpu')
model.eval()

In [ ]:
# Fuse model
modules_to_fuse = [
    ['encoder.enc_block.0', 'encoder.enc_block.1'],
    ['bottleneck.block.0', 'bottleneck.block.1'],
    ['decoder.dec_block.0', 'decoder.dec_block.1']
]
torch.quantization.fuse_modules(model, modules_to_fuse, inplace=True)

In [ ]:
# Quantize
model = torch.quantization.convert(model)

In [ ]:
# Trace model
example_input = torch.randn(1, 1, 128, 64)
traced_model = torch.jit.trace(model, example_input)

In [ ]:
# Optimize for mobile
optimized_model = torch.utils.mobile_optimizer.optimize_for_mobile(traced_model)

In [ ]:
torch.jit.save(optimized_model, "../_models/speech_denoiser_model_mobile.pt")